In [1]:
##################################################################################################################
# Libraries                                                                                                      #
##################################################################################################################
# import face_recognition as fr
import numpy as np
import pandas as pd
import cv2
import os
import pickle
import dlib
import time
import imutils
import datetime

from collections import Counter

import tkinter as tk
from tkinter.tix import *
from tkinter import *
from PIL import Image, ImageTk
from tkinter.filedialog import askopenfile
from tkinter import filedialog,messagebox


from IPython.display import display


from keras_facenet import FaceNet
from mtcnn.mtcnn import MTCNN
from deepface import DeepFace
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
import joblib



import warnings
warnings.filterwarnings('ignore')



In [25]:
##################################################################################################################
# Global Declaration variables                                                                                   #
##################################################################################################################
path = 'C:/Users/rajes/Documents/200 Images FaceNet/'
os.chdir(path)


#####################################################################
# Load Linear SVC Model
#####################################################################
model_path = 'C:/Users/rajes/Documents/200 Images FaceNet/FaceNet_LinearSVC_Model.sav'
loaded_model = joblib.load(model_path)
    

##################################################################################################################
# Start routine                                                                                                  #
##################################################################################################################
def start(): 
    
    root       = tk.Tk()
        
    w = 1100 # width for the Tk root
    h = 565 # height for the Tk root

    # get screen width and height
    ws = root.winfo_screenwidth() # width of the screen
    hs = root.winfo_screenheight() # height of the screen

    # calculate x and y coordinates for the Tk root window
    x = (ws/2) - (w/2)
    y = (hs/2) - (h/2)

    # set the dimensions of the screen and where it is placed
    root.geometry('%dx%d+%d+%d' % (w, h, x, y))                                 

    root.iconbitmap(r"C:/Users/rajes/Documents/Facematch/psx.ico")               ###SR
    Psx_img  = PhotoImage(file = r"C:/Users/rajes/Documents/Facematch/Psx1.png")  ###SR1
    root.title('Posidex Technologies') ###SR1
    
    #header area - logo & Title
    header = Frame(root, width=1100, height=150, bg="white")
    header.grid(columnspan=3, rowspan=2, row=0)

    head_title = Label(root,text = "FaceNet Linear SVC",bg ='#FFFFFF',fg= '#004488', font = ('Poppins',30,"bold"))
    head_title.place(x = 250,y=40) ###SR1
    

    
    headerPsx_lable = Label(root,text = 'Powered By ',bg ='#FFFFFF',fg= '#004488', font = ('Poppins',12,"bold"))###SR1 
    headerPsx_lable.place(x=0,y=0)###SR1
    headerPSX = Label(root,image = Psx_img,borderwidth=0) ###SR1
    headerPSX.place(x=0,y=20) ###SR1
        
    time = Label(root, text=f"{'{0:%d-%m-%Y %H:%M %p}'.format(datetime.datetime.now())}",bg ='#FFFFFF',fg= '#004488',
                 font = ('Poppins',12,"bold"))
    time.place(x = 450,y=-1)
        
    # MainFrame Area
    main_content = Frame(root,highlightbackground="#f80", highlightcolor="#f80", highlightthickness=2,width=1100, height=415, bg="#048")###SR
    main_content.grid(columnspan=3, rowspan=2, row=2)  
        


    ###############################################
    # Routine for Clear and Exit functionality    #
    ###############################################
                
    def Clear(): 
        root.destroy()
        start()
        
    def Exit(): 
        root.destroy()
        start()

    
    ##################################################################################################################
    # Routine to open the folder to select the file                                                                  #
    ##################################################################################################################

    def openfilename(): 
        
        global filepath
        
        filename = filedialog.askopenfilename(title = "Select File") 
        filepath = filename        
        return filename
        
    def clean(x):
        pattern = r'_[L?R].*/d+'
        result = re.sub(pattern,'',x)
        return result
    
    
    def Extract_Embeddings():
        
        global photo_path
        global Error_images
        global test_face_embeddings
        test_face_embeddings = {}
        Error_images = []
        
        photo_path  = openfilename() 
        img         = Image.open(photo_path)
        img         = img.resize((70,70)) 
        img         = ImageTk.PhotoImage(img) 
        cust_pict   = Label(root, image = img) 
        cust_pict.image    = img 
        cust_pict.place(x  = 30, y = 160)
        Upload_Button.place(anchor="nw", x=0, y=0, width=0, height=0) # Hide label & widget by setting the width and height to zero


        image = photo_path

        try:
            embeddings= DeepFace.represent(image,model_name='Facenet',enforce_detection=True)
            embeddings = np.asarray(embeddings)
            test_face_embeddings[image] = embeddings
            return test_face_embeddings
        except ValueError as Ve:
            pass
                
        
    def Compare_Faces():
        global test_face_embeddings
        
        Face_Embeddings = Extract_Embeddings()
        if Face_Embeddings:
            test_lables = list(Face_Embeddings.keys())
            test_encodings = list(Face_Embeddings.values())
            testX = np.asarray(test_encodings)
            testY = np.asarray(test_lables)
            Top_Preditions = loaded_model.decision_function(testX)
            pred_dic = dict(zip(loaded_model.classes_,Top_Preditions[0]))
            top10 = Counter(pred_dic)
            top10 = top10.most_common(10)
            images  =[x[0] for x in top10]
            
            test_img = testY[0].replace('test','train')
            if test_img in images:
                msg = Label(root,text = 'Face Matched',bg ="#048", fg="#FFFFFF",font=('Poppins',20,'bold'))
                msg.place(x  = 160 , y = 180)
                img         = Image.open(test_img)
                img         = img.resize((70,70)) 
                img         = ImageTk.PhotoImage(img)
                cust_pict   = Label(root, image = img) 
                cust_pict.image    = img 
                cust_pict.place(x  = 400, y = 160)
                
            else:
                msg = Label(root,text = 'Top 10 Preditions',bg ="#048", fg="#FFFFFF",font=('Poppins',22,'bold'))
                msg.place(x  = 400 , y = 280)
                x_val = 70
                for x in images[:5]:
                    img         = Image.open(x)
                    img         = img.resize((80,80)) 
                    img         = ImageTk.PhotoImage(img) 
                    Train_img   = Label(root, image = img) 
                    Train_img.image    = img 
                    Train_img.place(x  = x_val , y = 350)
                    x_val += 200
                x_val_2 = 70
                for x in images[5:]:
                    img         = Image.open(x)
                    img         = img.resize((80,80)) 
                    img         = ImageTk.PhotoImage(img) 
                    Train_img   = Label(root, image = img) 
                    Train_img.image    = img 
                    Train_img.place(x  = x_val_2 , y = 450)
                    x_val_2 += 200
        else:
            msg = Label(root,text = 'Please choose another image',bg ="#048", fg="#f80",font=('Poppins',22,'bold'))
            msg.place(x  = 160 , y = 180)


        


  
    Upload_Button = Button(root,command = Compare_Faces,text = "Upload Picture" ,font = ('Poppins',14),
                           relief=RAISED, bd = 6,bg ='#FFFFFF',fg= '#004488')
    Upload_Button.place(x = 10, y = 170,height = 40)   ##SR
    
    Exit_Button = Button(root, text="Exit",command=root.destroy,font = ('Poppins',14),relief=RAISED, bd = 6,
                         bg ='#FFFFFF',fg= '#004488')
    Exit_Button.place(x = 950, y =170,height = 40,width = 100)  ##SR
    
    Clear_Button   = Button(root, text="Clear",command=Clear,bg ='#FFFFFF',fg= '#004488',font = ('Poppins',14),
                            relief=RAISED,bd=6)
    Clear_Button.place(x = 800, y =170,height = 40,width = 100)  ##SR
     
    ###SR##########
    
    def Onbutton_upload(e):
        Upload_Button.config( bg ='#f80',fg= 'white')
    def Leavebutton_upload(e):
        Upload_Button.config( bg ='#FFFFFF',fg= '#004488')
    def Onbutton_Exit(e):
        Exit_Button.config( bg ='#f80',fg= 'white')
    def Leavebutton_Exit(e):
        Exit_Button.config( bg ='#FFFFFF',fg= '#004488')
    def Onbutton_Clear(e):
        Clear_Button.config( bg ='#f80',fg= 'white')
    def Leavebutton_Clear(e):
        Clear_Button.config( bg ='#FFFFFF',fg= '#004488')
    
    Upload_Button.bind('<Enter>',Onbutton_upload)
    Upload_Button.bind('<Leave>',Leavebutton_upload)
    Exit_Button.bind('<Enter>',Onbutton_Exit)
    Exit_Button.bind('<Leave>',Leavebutton_Exit)
    Clear_Button.bind('<Enter>',Onbutton_Clear)    ###SR
    Clear_Button.bind('<Leave>',Leavebutton_Clear) ###SR
    
    root.mainloop()
    
start()